In [7]:
import torch
import torch.nn as nn
import torch.nn.parallel
from __future__ import print_function
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import os

batchsize=64

class DCGAN_D(nn.Module):
    def __init__(self, isize=32, nz=100, nc=3, ndf=64, ngpu=0, n_extra_layers=0):
        super(DCGAN_D, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        main = nn.Sequential()
        # input is nc x isize x isize
        main.add_module('initial.conv.{0}-{1}'.format(nc, ndf),
                        nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        main.add_module('initial.relu.{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cndf),
                            nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cndf),
                            nn.BatchNorm2d(cndf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            main.add_module('pyramid.{0}-{1}.conv'.format(in_feat, out_feat),
                            nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(out_feat),
                            nn.BatchNorm2d(out_feat))
            main.add_module('pyramid.{0}.relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
        main.add_module('final.{0}-{1}.conv'.format(cndf, 1),
                        nn.Conv2d(cndf, 1, 4, 1, 0, bias=False))
        self.main = main


    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
            
        output = output.mean(0)
        return output.view(1)

class DCGAN_G(nn.Module):
    def __init__(self, isize=32, nz=100, nc=3, ngf=64, ngpu=0, n_extra_layers=0):
        super(DCGAN_G, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        cngf, tisize = ngf//2, 4
        while tisize != isize:
            cngf = cngf * 2
            tisize = tisize * 2

        main = nn.Sequential()
        # input is Z, going into a convolution
        main.add_module('initial.{0}-{1}.convt'.format(nz, cngf),
                        nn.ConvTranspose2d(nz, cngf, 4, 1, 0, bias=False))
        main.add_module('initial.{0}.batchnorm'.format(cngf),
                        nn.BatchNorm2d(cngf))
        main.add_module('initial.{0}.relu'.format(cngf),
                        nn.ReLU(True))

        csize, cndf = 4, cngf
        while csize < isize//2:
            main.add_module('pyramid.{0}-{1}.convt'.format(cngf, cngf//2),
                            nn.ConvTranspose2d(cngf, cngf//2, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(cngf//2),
                            nn.BatchNorm2d(cngf//2))
            main.add_module('pyramid.{0}.relu'.format(cngf//2),
                            nn.ReLU(True))
            cngf = cngf // 2
            csize = csize * 2

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cngf),
                            nn.Conv2d(cngf, cngf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cngf),
                            nn.BatchNorm2d(cngf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cngf),
                            nn.ReLU(True))

        main.add_module('final.{0}-{1}.convt'.format(cngf, nc),
                        nn.ConvTranspose2d(cngf, nc, 4, 2, 1, bias=False))
        main.add_module('final.{0}.tanh'.format(nc),
                        nn.Tanh())
        self.main = main

    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
        return output 
###############################################################################

In [ ]:
import cv2
import numpy as np
import functions
batchSize = 64

X_train = functions.get_mnist()
X_label = torch.LongTensor(np.zeros((X_train.shape[0]),dtype=int))
X_train = torch.FloatTensor(X_train)
train = torch.utils.data.TensorDataset(X_train,X_label)
dataloader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=batchsize)

dataiter = iter(dataloader)
ngpu = 0
nz = 100
ngf = 64
ndf = 64
nc = 3
n_extra_layers = 0

# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

netG = DCGAN_G()
netG.apply(weights_init)
print(netG)

netD = DCGAN_D()
netD.apply(weights_init)
print(netD)

input = torch.FloatTensor(batchSize, 3, 32, 32)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)
one = torch.FloatTensor([1])
mone = one * -1

netD.cuda()
netG.cuda()
input = input.cuda()
one, mone = one.cuda(), mone.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

optimizerD = optim.RMSprop(netD.parameters(), lr = 0.00005)
optimizerG = optim.RMSprop(netG.parameters(), lr = 0.00005)

gen_iterations = 0
for epoch in range(10000):
    data_iter = iter(dataloader)
    i = 0
    while i < len(dataloader):
        ############################
        # (1) Update D network
        ###########################
        for p in netD.parameters(): # reset requires_grad
            p.requires_grad = True # they are set to False below in netG update

        # train the discriminator Diters times
        if gen_iterations < 25 or gen_iterations % 500 == 0:
            Diters = 100
        else:
            Diters = 5
        j = 0
        while j < Diters and i < len(dataloader):
            j += 1

            # clamp parameters to a cube
            for p in netD.parameters():
                p.data.clamp_(-0.01, 0.01)

            data = data_iter.next()
            i += 1

            # train with real
            real_cpu, _ = data
            netD.zero_grad()
            batch_size = real_cpu.size(0)

            real_cpu = real_cpu.cuda()
            input.resize_as_(real_cpu).copy_(real_cpu)
            inputv = Variable(input)

            errD_real = netD(inputv)
            errD_real.backward(one)

            # train with fake
            noise.resize_(64, 100, 1, 1).normal_(0, 1)
            noisev = Variable(noise, volatile = True) # totally freeze netG
            fake = Variable(netG(noisev).data)
            inputv = fake
            errD_fake = netD(inputv)
            errD_fake.backward(mone)
            errD = errD_real - errD_fake
            optimizerD.step()

        ############################
        # (2) Update G network
        ###########################
        for p in netD.parameters():
            p.requires_grad = False # to avoid computation
        netG.zero_grad()
        # in case our last batch was the tail batch of the dataloader,
        # make sure we feed a full batch of noise
        noise.resize_(64, 100, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        errG = netD(fake)
        errG.backward(one)
        optimizerG.step()
        gen_iterations += 1


        if gen_iterations % 20 == 0:
            
            print('[%d/%d][%d/%d][%d] Loss_D: %f Loss_G: %f Loss_D_real: %f Loss_D_fake %f'
            % (epoch, 10000, i, len(dataloader), gen_iterations,
            errD.data[0], errG.data[0], errD_real.data[0], errD_fake.data[0]))
            real_cpu = real_cpu.mul(0.5).add(0.5)
            vutils.save_image(real_cpu, 'real_samples_kmeans.png')
            fake = netG(Variable(fixed_noise, volatile=True))
            fake.data = fake.data.mul(0.5).add(0.5)
            vutils.save_image(fake.data, 'fake_samples_kmeans.png')

DCGAN_G (
  (main): Sequential (
    (initial.100-256.convt): ConvTranspose2d(100, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (initial.256.batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (initial.256.relu): ReLU (inplace)
    (pyramid.256-128.convt): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.128.batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.128.relu): ReLU (inplace)
    (pyramid.128-64.convt): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.64.batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.64.relu): ReLU (inplace)
    (final.64-3.convt): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (final.3.tanh): Tanh ()
  )
)
DCGAN_D (
  (main): Sequential (
    (initial.conv.3-64): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)

[9/10000][485/938][1300] Loss_D: -0.476959 Loss_G: 0.232159 Loss_D_real: -0.240847 Loss_D_fake 0.236112
[9/10000][585/938][1320] Loss_D: -0.477710 Loss_G: 0.229520 Loss_D_real: -0.231136 Loss_D_fake 0.246575
[9/10000][685/938][1340] Loss_D: -0.484646 Loss_G: 0.218132 Loss_D_real: -0.230776 Loss_D_fake 0.253870
[9/10000][785/938][1360] Loss_D: -0.379710 Loss_G: 0.208154 Loss_D_real: -0.220399 Loss_D_fake 0.159311
[9/10000][885/938][1380] Loss_D: -0.443400 Loss_G: 0.210744 Loss_D_real: -0.218073 Loss_D_fake 0.225326
[10/10000][45/938][1400] Loss_D: -0.473885 Loss_G: 0.227550 Loss_D_real: -0.240045 Loss_D_fake 0.233840
[10/10000][145/938][1420] Loss_D: -0.490333 Loss_G: 0.211276 Loss_D_real: -0.236230 Loss_D_fake 0.254104
[10/10000][245/938][1440] Loss_D: -0.443174 Loss_G: 0.161288 Loss_D_real: -0.178664 Loss_D_fake 0.264510
[10/10000][345/938][1460] Loss_D: -0.426641 Loss_G: 0.138182 Loss_D_real: -0.160672 Loss_D_fake 0.265969
[10/10000][445/938][1480] Loss_D: -0.458628 Loss_G: 0.223763 

[18/10000][210/938][2880] Loss_D: -0.342895 Loss_G: 0.138388 Loss_D_real: -0.138960 Loss_D_fake 0.203935
[18/10000][310/938][2900] Loss_D: -0.287441 Loss_G: 0.071174 Loss_D_real: -0.078865 Loss_D_fake 0.208577
[18/10000][410/938][2920] Loss_D: -0.328022 Loss_G: 0.105779 Loss_D_real: -0.138659 Loss_D_fake 0.189363
[18/10000][510/938][2940] Loss_D: -0.304795 Loss_G: 0.219716 Loss_D_real: -0.206380 Loss_D_fake 0.098415
[18/10000][610/938][2960] Loss_D: -0.317361 Loss_G: 0.113667 Loss_D_real: -0.098540 Loss_D_fake 0.218822
[18/10000][710/938][2980] Loss_D: -0.291696 Loss_G: 0.209178 Loss_D_real: -0.187557 Loss_D_fake 0.104140
[18/10000][810/938][3000] Loss_D: -0.254627 Loss_G: 0.082640 Loss_D_real: -0.043366 Loss_D_fake 0.211261
[19/10000][65/938][3020] Loss_D: -0.292495 Loss_G: 0.115632 Loss_D_real: -0.067053 Loss_D_fake 0.225442
[19/10000][165/938][3040] Loss_D: -0.314537 Loss_G: 0.081918 Loss_D_real: -0.107346 Loss_D_fake 0.207191
[19/10000][265/938][3060] Loss_D: -0.323807 Loss_G: 0.22

[26/10000][875/938][4460] Loss_D: -0.304408 Loss_G: 0.169586 Loss_D_real: -0.177203 Loss_D_fake 0.127205
[27/10000][35/938][4480] Loss_D: -0.299452 Loss_G: 0.056154 Loss_D_real: -0.078609 Loss_D_fake 0.220843
[27/10000][135/938][4500] Loss_D: -0.295938 Loss_G: 0.089904 Loss_D_real: -0.093338 Loss_D_fake 0.202601
[27/10000][330/938][4520] Loss_D: -0.239713 Loss_G: 0.208686 Loss_D_real: -0.228095 Loss_D_fake 0.011618
[27/10000][430/938][4540] Loss_D: -0.280052 Loss_G: 0.222958 Loss_D_real: -0.221954 Loss_D_fake 0.058098
[27/10000][530/938][4560] Loss_D: -0.281222 Loss_G: 0.218947 Loss_D_real: -0.182536 Loss_D_fake 0.098687
[27/10000][630/938][4580] Loss_D: -0.309773 Loss_G: 0.178567 Loss_D_real: -0.161347 Loss_D_fake 0.148426
[27/10000][730/938][4600] Loss_D: -0.270504 Loss_G: 0.214640 Loss_D_real: -0.194916 Loss_D_fake 0.075588
[27/10000][830/938][4620] Loss_D: -0.240812 Loss_G: 0.211803 Loss_D_real: -0.185230 Loss_D_fake 0.055582
[27/10000][930/938][4640] Loss_D: -0.270982 Loss_G: 0.08

[35/10000][685/938][6040] Loss_D: -0.300737 Loss_G: -0.009913 Loss_D_real: -0.117633 Loss_D_fake 0.183104
[35/10000][785/938][6060] Loss_D: -0.258079 Loss_G: 0.012035 Loss_D_real: -0.064489 Loss_D_fake 0.193590
[35/10000][885/938][6080] Loss_D: -0.294342 Loss_G: 0.047229 Loss_D_real: -0.087325 Loss_D_fake 0.207017
[36/10000][45/938][6100] Loss_D: -0.242424 Loss_G: 0.089708 Loss_D_real: -0.044032 Loss_D_fake 0.198392
[36/10000][145/938][6120] Loss_D: -0.250571 Loss_G: 0.011971 Loss_D_real: -0.043511 Loss_D_fake 0.207060
[36/10000][245/938][6140] Loss_D: -0.259466 Loss_G: 0.017955 Loss_D_real: -0.057815 Loss_D_fake 0.201652
[36/10000][345/938][6160] Loss_D: -0.261743 Loss_G: 0.101768 Loss_D_real: -0.076795 Loss_D_fake 0.184948
[36/10000][445/938][6180] Loss_D: -0.279473 Loss_G: 0.070739 Loss_D_real: -0.093593 Loss_D_fake 0.185880
[36/10000][545/938][6200] Loss_D: -0.200872 Loss_G: 0.004897 Loss_D_real: -0.011115 Loss_D_fake 0.189757
[36/10000][645/938][6220] Loss_D: -0.247608 Loss_G: 0.2

...

[683/10000][30/938][123740] Loss_D: -0.102124 Loss_G: 0.164984 Loss_D_real: -0.095220 Loss_D_fake 0.006904
[683/10000][130/938][123760] Loss_D: -0.093911 Loss_G: 0.111254 Loss_D_real: -0.104767 Loss_D_fake -0.010856
[683/10000][230/938][123780] Loss_D: -0.104057 Loss_G: 0.031178 Loss_D_real: -0.057522 Loss_D_fake 0.046535
[683/10000][330/938][123800] Loss_D: -0.102367 Loss_G: 0.110714 Loss_D_real: -0.119290 Loss_D_fake -0.016924
[683/10000][430/938][123820] Loss_D: -0.095362 Loss_G: -0.022574 Loss_D_real: -0.018998 Loss_D_fake 0.076364
[683/10000][530/938][123840] Loss_D: -0.079521 Loss_G: 0.118183 Loss_D_real: -0.155547 Loss_D_fake -0.076025
[683/10000][630/938][123860] Loss_D: -0.086948 Loss_G: 0.078826 Loss_D_real: -0.111802 Loss_D_fake -0.024854
[683/10000][730/938][123880] Loss_D: -0.091519 Loss_G: 0.002961 Loss_D_real: -0.017135 Loss_D_fake 0.074383
[683/10000][830/938][123900] Loss_D: -0.081332 Loss_G: -0.025156 Loss_D_real: 0.022039 Loss_D_fake 0.103371
[683/10000][930/938][123

[691/10000][355/938][125260] Loss_D: -0.058575 Loss_G: 0.004506 Loss_D_real: -0.123181 Loss_D_fake -0.064606
[691/10000][455/938][125280] Loss_D: -0.086555 Loss_G: 0.125751 Loss_D_real: -0.122511 Loss_D_fake -0.035956
[691/10000][555/938][125300] Loss_D: -0.093010 Loss_G: 0.075616 Loss_D_real: -0.093758 Loss_D_fake -0.000748
[691/10000][655/938][125320] Loss_D: -0.078583 Loss_G: 0.099342 Loss_D_real: -0.136517 Loss_D_fake -0.057933
[691/10000][755/938][125340] Loss_D: -0.079421 Loss_G: 0.104206 Loss_D_real: -0.139294 Loss_D_fake -0.059873
[691/10000][855/938][125360] Loss_D: -0.111968 Loss_G: 0.066624 Loss_D_real: -0.085783 Loss_D_fake 0.026186
[692/10000][15/938][125380] Loss_D: -0.085264 Loss_G: 0.129395 Loss_D_real: -0.152786 Loss_D_fake -0.067522
[692/10000][115/938][125400] Loss_D: -0.090221 Loss_G: 0.098418 Loss_D_real: -0.129926 Loss_D_fake -0.039704
[692/10000][215/938][125420] Loss_D: -0.084652 Loss_G: 0.061530 Loss_D_real: -0.126311 Loss_D_fake -0.041659
[692/10000][315/938][

[699/10000][720/938][126780] Loss_D: -0.073189 Loss_G: 0.002994 Loss_D_real: 0.001617 Loss_D_fake 0.074806
[699/10000][820/938][126800] Loss_D: -0.094053 Loss_G: 0.033897 Loss_D_real: -0.016494 Loss_D_fake 0.077559
[699/10000][920/938][126820] Loss_D: -0.079791 Loss_G: 0.154122 Loss_D_real: -0.084838 Loss_D_fake -0.005047
[700/10000][80/938][126840] Loss_D: -0.073572 Loss_G: 0.057286 Loss_D_real: -0.116371 Loss_D_fake -0.042799
[700/10000][180/938][126860] Loss_D: -0.069491 Loss_G: -0.050309 Loss_D_real: 0.069658 Loss_D_fake 0.139150
[700/10000][280/938][126880] Loss_D: -0.084755 Loss_G: 0.078509 Loss_D_real: -0.130414 Loss_D_fake -0.045659
[700/10000][380/938][126900] Loss_D: -0.089553 Loss_G: -0.018668 Loss_D_real: -0.041366 Loss_D_fake 0.048188
[700/10000][480/938][126920] Loss_D: -0.087501 Loss_G: 0.068960 Loss_D_real: -0.076980 Loss_D_fake 0.010522
[700/10000][580/938][126940] Loss_D: -0.111465 Loss_G: 0.010579 Loss_D_real: -0.040588 Loss_D_fake 0.070877
[700/10000][680/938][12696

[708/10000][105/938][128300] Loss_D: -0.093584 Loss_G: 0.109365 Loss_D_real: -0.144127 Loss_D_fake -0.050543
[708/10000][205/938][128320] Loss_D: -0.088053 Loss_G: 0.011529 Loss_D_real: -0.015039 Loss_D_fake 0.073014
[708/10000][305/938][128340] Loss_D: -0.092586 Loss_G: 0.001078 Loss_D_real: -0.015525 Loss_D_fake 0.077061
[708/10000][405/938][128360] Loss_D: -0.097928 Loss_G: 0.030053 Loss_D_real: -0.060435 Loss_D_fake 0.037493
[708/10000][505/938][128380] Loss_D: -0.091057 Loss_G: 0.020892 Loss_D_real: -0.045172 Loss_D_fake 0.045885
[708/10000][605/938][128400] Loss_D: -0.113083 Loss_G: -0.000413 Loss_D_real: -0.044630 Loss_D_fake 0.068453
[708/10000][705/938][128420] Loss_D: -0.081004 Loss_G: 0.073306 Loss_D_real: -0.123875 Loss_D_fake -0.042870
[708/10000][805/938][128440] Loss_D: -0.081949 Loss_G: 0.089619 Loss_D_real: -0.117600 Loss_D_fake -0.035651
[708/10000][905/938][128460] Loss_D: -0.096962 Loss_G: 0.085287 Loss_D_real: -0.098209 Loss_D_fake -0.001247
[709/10000][65/938][128

[716/10000][430/938][129820] Loss_D: -0.083564 Loss_G: 0.078143 Loss_D_real: -0.141213 Loss_D_fake -0.057649
[716/10000][530/938][129840] Loss_D: -0.069235 Loss_G: 0.001196 Loss_D_real: -0.015722 Loss_D_fake 0.053513
[716/10000][630/938][129860] Loss_D: -0.093749 Loss_G: -0.029558 Loss_D_real: 0.010294 Loss_D_fake 0.104044
[716/10000][730/938][129880] Loss_D: -0.102690 Loss_G: 0.029401 Loss_D_real: -0.085409 Loss_D_fake 0.017281
[716/10000][830/938][129900] Loss_D: -0.070886 Loss_G: 0.038180 Loss_D_real: -0.093983 Loss_D_fake -0.023097
[716/10000][930/938][129920] Loss_D: -0.112052 Loss_G: 0.087483 Loss_D_real: -0.073231 Loss_D_fake 0.038821
[717/10000][90/938][129940] Loss_D: -0.088077 Loss_G: 0.073920 Loss_D_real: -0.115374 Loss_D_fake -0.027297
[717/10000][190/938][129960] Loss_D: -0.105637 Loss_G: 0.006051 Loss_D_real: -0.016431 Loss_D_fake 0.089206
[717/10000][290/938][129980] Loss_D: -0.094426 Loss_G: 0.111992 Loss_D_real: -0.116768 Loss_D_fake -0.022342
[717/10000][390/938][1300

In [9]:
torch.save(netG.state_dict(), 'netG_try_kmeans_epoch_%d.pth' % (epoch))
torch.save(netD.state_dict(), 'netD_try_kmeans_epoch_%d.pth' % (epoch))

In [20]:
class DCGAN_D_feature(nn.Module):
    def __init__(self, isize=32, nz=100, nc=3, ndf=64, ngpu=0, n_extra_layers=0):
        super(DCGAN_D_feature, self).__init__()
        self.ngpu = ngpu
        assert isize % 16 == 0, "isize has to be a multiple of 16"

        main = nn.Sequential()
        # input is nc x isize x isize
        main.add_module('initial.conv.{0}-{1}'.format(nc, ndf),
                        nn.Conv2d(nc, ndf, 4, 2, 1, bias=False))
        main.add_module('initial.relu.{0}'.format(ndf),
                        nn.LeakyReLU(0.2, inplace=True))
        csize, cndf = isize / 2, ndf

        # Extra layers
        for t in range(n_extra_layers):
            main.add_module('extra-layers-{0}.{1}.conv'.format(t, cndf),
                            nn.Conv2d(cndf, cndf, 3, 1, 1, bias=False))
            main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, cndf),
                            nn.BatchNorm2d(cndf))
            main.add_module('extra-layers-{0}.{1}.relu'.format(t, cndf),
                            nn.LeakyReLU(0.2, inplace=True))

        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            main.add_module('pyramid.{0}-{1}.conv'.format(in_feat, out_feat),
                            nn.Conv2d(in_feat, out_feat, 4, 2, 1, bias=False))
            main.add_module('pyramid.{0}.batchnorm'.format(out_feat),
                            nn.BatchNorm2d(out_feat))
            main.add_module('pyramid.{0}.relu'.format(out_feat),
                            nn.LeakyReLU(0.2, inplace=True))
            cndf = cndf * 2
            csize = csize / 2

        # state size. K x 4 x 4
        #main.add_module('final.{0}-{1}.conv'.format(cndf, 1),
                        #nn.Conv2d(cndf, 1, 4, 1, 0, bias=False))
        self.main = main


    def forward(self, input):
        if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else: 
            output = self.main(input)
            
        #output = output.mean(0)
        return output

In [50]:
import cv2
import numpy as np
batchSize = 1

X_train = functions.get_mnist()
X_train = X_train[0:1000,:,:,:]
X_label = torch.LongTensor(np.zeros((X_train.shape[0]),dtype=int))
X_train_ = torch.FloatTensor(X_train)
train = torch.utils.data.TensorDataset(X_train_,X_label)
dataloader = torch.utils.data.DataLoader(train, shuffle=False, batch_size=batchSize)

dataiter = iter(dataloader)
ngpu = 0
nz = 100
ngf = 64
ndf = 64
nc = 3
n_extra_layers = 0

In [23]:
dict_G = torch.load('netG_try_kmeans_epoch_%d.pth' % (1910))
dict_D = torch.load('netD_try_kmeans_epoch_%d.pth' % (1910))
del(dict_D['main.final.256-1.conv.weight'])

feature_dict = {}
netD_feature = DCGAN_D_feature()
netD_feature.load_state_dict(dict_D)
print(netD_feature)
data_iter = iter(dataloader)
for i,data in enumerate(data_iter):
    feature_dict[i] = netD_feature(Variable(data[0])).data.numpy().reshape((1,256*4*4))
    
feature = np.zeros((max(feature_dict.keys()),4096),dtype=np.float32)
for i,data in enumerate(feature):
    data[:] = feature_dict[i] 

DCGAN_D_feature (
  (main): Sequential (
    (initial.conv.3-64): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (initial.relu.64): LeakyReLU (0.2, inplace)
    (pyramid.64-128.conv): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.128.batchnorm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.128.relu): LeakyReLU (0.2, inplace)
    (pyramid.128-256.conv): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (pyramid.256.batchnorm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (pyramid.256.relu): LeakyReLU (0.2, inplace)
  )
)


In [26]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=0).fit_predict(feature)

In [64]:
arg = [np.argwhere(kmeans==n) for n in range(0,10)]
for i in range(0,10):
    cluster = np.take(X_train, arg[i],axis=0).reshape(-1,3,32,32)
    vutils.save_image(torch.from_numpy(cluster), 'cluster%d_%d.png'% (10,i),normalize=True)

In [59]:
X_train.shape

(1000, 3, 32, 32)

In [55]:
np.take(X_train, [0],axis=0).shape

(1, 3, 32, 32)

In [46]:
X_train[6]


(0 ,.,.) = 
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
           ...             ⋱             ...          
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000

(1 ,.,.) = 
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
           ...             ⋱             ...          
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000

(2 ,.,.) = 
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
 -1.0000 -1.0000 -1.0000  ...  -1.0000 -1.0000 -1.0000
           ...            

In [32]:
arg = [np.argwhere(kmeans==n) for n in range(0,10)]
pic_list = []
for i,index in enumerate(arg):
    X_train[n,:,:,:]


[array([[  0],
        [  6],
        [ 20],
        [ 24],
        [ 37],
        [ 42],
        [ 49],
        [ 51],
        [ 59],
        [111],
        [132],
        [134],
        [136],
        [152],
        [159],
        [188],
        [193],
        [204],
        [214],
        [231],
        [236],
        [241],
        [245],
        [255],
        [269],
        [274],
        [276],
        [289],
        [296],
        [300],
        [308],
        [329],
        [333],
        [336],
        [340],
        [342],
        [345],
        [363],
        [372],
        [395],
        [409],
        [447],
        [475],
        [479],
        [505],
        [508],
        [516],
        [520],
        [522],
        [530],
        [554],
        [567],
        [569],
        [573],
        [580],
        [588],
        [602],
        [612],
        [613],
        [615],
        [627],
        [629],
        [648],
        [658],
        [683],
        [696],
        [6